<a href="https://colab.research.google.com/github/OlesiaPoliakova/Education/blob/main/MaintinableScripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installing dependencies
!pip install pytest ipython_pytest ipyext unittest2 requests pandas
%load_ext ipython_pytest
%load_ext ipyext.writeandexecute

!rm *.py
!rm locations.csv

rows = [
        "kyiv,ua",
        "warsaw,pl",
        "berlin,de"
]

with open("locations.csv", "w") as locations:
  for row in rows:
    locations.write(row + "\n")

#Maintainable Test Scripts Approaches
##Incapsulate SUT interaction **beings** into **classes**. Create *REST API Client* for [сайту Open Weather](https://openweathermap.org/api) 
###Create **helper** to properly form *REST Query URL*

In [ ]:
%%writeandexecute -i identifier example_helper.py

def get_openweather_api_query_url(endpoint, api_key, **kwargs):
  req_args = dict(kwargs)
  return f"https://api.openweathermap.org/data/2.5/{endpoint}?{'&'.join([f'{i}={j}' for i, j in zip(req_args.keys(), req_args.values())])}&appid={api_key}"

###Create Abstract API Client to maintain many kinds of clients in the framework

In [ ]:
%%writeandexecute -i identifier example_abstract_api_client.py

import json
import requests
from example_helper import *

class AbstractApiClient:
    def __init__(self, api_key):
        self.__api_key = api_key
        self.__client = requests.Session()

    @property
    def api_key(self):
        return self.__api_key

    def _get(self, endpoint, **kwargs):
        url = get_openweather_api_query_url(endpoint, self.__api_key, **kwargs)
        res = self.__client.get(url)
        return res

    def __del__(self):
        self.__client.close()

###Create Client for some kind of features (get current weather for some options)

In [ ]:
%%writeandexecute -i identifier example_current_weather_client.py

from example_abstract_api_client import AbstractApiClient

class CurrentWeatherClient(AbstractApiClient):

    def get_current_weather(self, city, country_code=None, state_code=None):
        q = city
        if state_code:
            q += f",{state_code}"
        if country_code:
            q += f",{country_code}"

        return self._get('weather', q=q)

## To get current weather client into our test without lowlevel code in the test, create *test fixture*

In [ ]:
%%writeandexecute -i identifier example_fixtures.py

import pytest
from example_current_weather_client import CurrentWeatherClient

@pytest.fixture(scope="session")
def current_weather_client():
    api_client = CurrentWeatherClient(api_key="15bca8c10e86424df00fa826657dca87")
    yield api_client
    del api_client

###Create necessary *test hooks* to parametrize tests

In [ ]:
%%writeandexecute -i identifier example_test_hooks.py

def pytest_generate_tests(metafunc):
    rows = (
         ("kyiv", "ua"),
         ("warsaw", "pl"),
         ("berlin", "de")
     )

    if "current_weather_client" in metafunc.fixturenames:
        metafunc.parametrize("city, country_code", rows)

##Now we can create tests which can be maintained

In [ ]:
%%pytest

from example_fixtures import *
from example_test_hooks import *

class TestCurrentWeather:
    @pytest.mark.city_country
    def test_current_weather_sync(self, current_weather_client: CurrentWeatherClient, city, country_code):
        single_city_request = current_weather_client.get_current_weather(city)
        city_country_request = current_weather_client.get_current_weather(city, country_code=country_code)

        assert single_city_request.ok, f"Request failed with the status code {single_city_request.status_code}"
        assert city_country_request.ok, f"Request failed with the status code {city_country_request.status_code}"

        assert single_city_request.json() == city_country_request.json(), f"""Requests with city and city with country give different results
        With city ({city}) only
        {single_city_request.json()}
        With city and country ({city}, {country_code})
        {city_country_request.json()}"""